## Next book you should read

This code will given a favorite book/bookshelf read off goodreads API to find users with similar bookshelfs to form recommendations

### Import all relavent libraries

In [1]:
import pandas as pd
import rauth, os, urllib2, requests, re, time
from rauth.service import OAuth1Service, OAuth1Session
import xml.etree.ElementTree as ET ## Parse XML stuff
from selenium import webdriver ## to get around iframe stuff on goodreads website
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from bs4 import BeautifulSoup
from collections import defaultdict
import numpy as np


### Talk to goodreads API


In [2]:
## Blindly copied stuff from goodreads website
def accessGoodreads():
    goodreadsSession = OAuth1Session(
        consumer_key = os.environ['goodreads_key'],
        consumer_secret = os.environ['goodreads_secret'],
        access_token = os.environ['goodreads_access_token'],
        access_token_secret = os.environ['goodreads_access_token_secret'])
    return goodreadsSession
    
session = accessGoodreads()

### Search for a book and get a response

In [3]:
bookname = "Harry Potter"
bookname = urllib2.quote(bookname)
def getBookID(bookname):
    ## Get web page for searching a book
    url = \
        "https://www.goodreads.com/search/index.xml?key=%s&q=%s" %\
        (os.environ['goodreads_key'], bookname)
    response = session.get(url)

    ## Trust the first response to be the right response?
    page = response.content
    root = ET.fromstring(page.encode('UTF-8'))
    books = root[1]

    ## Extract book id
    return root.find('search').find('results').find('work').find('best_book').find('id').text,root.find('search').find('results').find('work').find('best_book').find('title').text
#bookID = getBookID(bookname)

### Extract user info of all the people who read this book

In [4]:
bookID, bookName = getBookID(bookname)

In [ ]:
browser = webdriver.Chrome("chromedriver")
def getUserInfo(df, bookID, bookName, browser, page):
    url = \
        "https://www.goodreads.com/book/show/%s?page=%s" % (bookID, page)
    browser.get(url)
    time.sleep(2.0)
    bookPage = browser.page_source
    soup = BeautifulSoup(bookPage, "html.parser")
    letters = soup.findAll('div', class_='left bodycol')
    reviews = defaultdict(list)
    for element in letters:
        reviews['user_link'].append(element.find('a', class_='user').get('href'))
        if element.find('span', class_ = 'staticStar p10'):
            reviews['rating'].append(element.find('span', class_ = 'staticStar p10').get('title'))
        else:
            reviews['rating'].append('')
        reviews['title'].append(bookName)
    reviewsDf = pd.DataFrame.from_dict(reviews)
    df = pd.concat([df, reviewsDf])
   # print len(reviewsDf)
    if len(reviewsDf)<31:
        return df
    else:
        time.sleep(2.0)
        return getUserInfo(df, bookID, bookName, browser, page+1)

#df = pd.DataFrame(columns = {'title', 'user_link', 'rating'})
#reviewsDf = getUserInfo(df, bookID, bookName, browser, 111)
#reviewsDf.to_pickle('goodReadsDataset')


### Read shelves off users!

In [12]:
### Reduce the dataset to only those who really really like the book
reviewsDf=pd.read_pickle('goodReadsDataset')
reviewsDf = reviewsDf[(reviewsDf['rating'] == 'it was amazing') | (reviewsDf['rating'] == 'really liked it')]
### Go to all the users shelves who read your favorite book and extract their favorite books
reviewsDf['userID'] = reviewsDf['user_link'].str.split('/').str.get(3)
userIDs = set(reviewsDf['userID'])
userIDs = set(list(userIDs)[750:1000])

## Depth first traversal 
def aggCommonRatings(df, userIDs, page, visiting=None):
    if visiting is None:
        visiting = set()
        if len(userIDs) == 0:
            return df
        else:
            userID = userIDs.pop()
            visiting.add(userID)
    
    time.sleep(1.0)
    url = "https://www.goodreads.com/review/list/%s.xml?key=%s&v=2?shelf=read&per_page=200&page=%s&sort=rating" %\
        (str(list(visiting)[0]), os.environ['goodreads_key'],page)
    response = session.get(url)
    time.sleep(1.0)
    webpage = response.content
    root = ET.fromstring(webpage)
    if len(root) != 0 or root.find('error') is not None:
        print "no error", url
        reviews = defaultdict(list)
        something = root.find('reviews')
        
        for i in range(0,len(something)):
            reviews['Title'].append(something[i][1][4].text)
            reviews['Author'].append(something[i][1][19][0][1].text)
            reviews['Rating'].append(something[i][2].text)
            reviews['userName'].append(list(visiting)[0])
            reviews['avgRating'].append(something[i][1][16].text)
            reviews['reviewCount'].append(something[i][1][17].text)
        
        reviewsDf = pd.DataFrame.from_dict(reviews)
        df = pd.concat([df, reviewsDf])
        #df.to_pickle('goodreadsUserRatings2')
            
        if (len(reviewsDf)<199) | (np.min(reviewsDf.Rating) == '0'):
            if len(userIDs) == 0:
                return df
            else:
                print len(df), str(list(visiting)[0])
                return aggCommonRatings(df, userIDs, 1, visiting = None)
        else:
            time.sleep(1.0)
            return aggCommonRatings(df, userIDs, page+1, visiting)
    else:
        print 'error', list(visiting)[0], url
        return aggCommonRatings(df, userIDs, 1, visiting = None)



initDf = pd.DataFrame(columns = ('Title', 'Rating', 'userName', 'Author', 'avgRating', 'reviewCount'))
summaryDf = aggCommonRatings(initDf, userIDs, 1)
summaryDf.to_pickle('goodreadsUserRatings7')

error 1563349-olivia https://www.goodreads.com/review/list/1563349-olivia.xml?key=r5bgUC0mnj9A2HSufgT3g&v=2?shelf=read&per_page=200&page=1&sort=rating
no error https://www.goodreads.com/review/list/28351026-tally.xml?key=r5bgUC0mnj9A2HSufgT3g&v=2?shelf=read&per_page=200&page=1&sort=rating
200 28351026-tally
error 18562741-jacquelyn https://www.goodreads.com/review/list/18562741-jacquelyn.xml?key=r5bgUC0mnj9A2HSufgT3g&v=2?shelf=read&per_page=200&page=1&sort=rating
no error https://www.goodreads.com/review/list/16287407-carly-patrick.xml?key=r5bgUC0mnj9A2HSufgT3g&v=2?shelf=read&per_page=200&page=1&sort=rating
365 16287407-carly-patrick
error 3431708-hannahcassie https://www.goodreads.com/review/list/3431708-hannahcassie.xml?key=r5bgUC0mnj9A2HSufgT3g&v=2?shelf=read&per_page=200&page=1&sort=rating
no error https://www.goodreads.com/review/list/28520980-sammy-jackson.xml?key=r5bgUC0mnj9A2HSufgT3g&v=2?shelf=read&per_page=200&page=1&sort=rating
561 28520980-sammy-jackson
no error https://www.

In [6]:
len(initDf)




0

In [ ]:
a = root.find('reviews')
print a[0][1][17].text


In [ ]:
summaryDf.head()

In [ ]:


userIDs = list(set(reviewsDf['userID']))
url = "https://www.goodreads.com/review/list/%s.xml?key=%s&v=2?shelf=read&per_page=50&page=%s"%\
(str(userIDs[1]), os.environ['goodreads_key'],1)
response = session.get(url)
time.sleep(1.0)
webpage = response.content
if root.find('error') is None:
    a = root.find('reviews')
    i=0
    something = defaultdict(list)
    for node in a:
        something['something'].append(a[i][1][4].text)
        print a[i][1][4].text, len(node)
        i=i+1
    

In [ ]:
userIDs = list(set(reviewsDf['userID']))
url = "https://www.goodreads.com/review/list/%s.xml?key=%s&v=2?shelf=read&per_page=50&page=%s" %\
(str(userIDs[1]), os.environ['goodreads_key'],1)

In [ ]:
reviewsDf=pd.read_pickle('goodReadsDataset')
reviewsDf = reviewsDf[(reviewsDf['rating'] == 'it was amazing') | (reviewsDf['rating'] == 'really liked it')]
### Go to all the users shelves who read your favorite book and extract their favorite books
reviewsDf['userID'] = reviewsDf['user_link'].str.split('/').str.get(3)
userIDs = set(reviewsDf['userID'])


In [ ]:
list(userIDs)[252]


In [ ]:
len(userIDs)

In [ ]:
a=[1,2]

In [ ]:
print a.pop(), len(a)

In [ ]:
reviewsDf = reviewsDf[(reviewsDf['rating'] == 'it was amazing')]

In [ ]:
len(reviewsDf)

In [ ]:
newDf = pd.read_pickle('goodreadsUserRatings2')

In [ ]:
len(set(newDf.userName))